In [30]:
import urllib3
from zipfile import ZipFile
from io import StringIO
import shapefile
import geopandas as gpd
from shapely.geometry import shape  
import osr
import pandas as pd
import requests
import matplotlib.pyplot as plt
%matplotlib inline

#specify zipped shapefile url
county_file_url = 'http://www2.census.gov/geo/tiger/GENZ2015/shp/cb_2015_us_county_500k.zip'

zipfile = ZipFile(StringIO(request(county_file_url).read()))

filenames = [y for y in sorted(zipfile.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
print(filenames)

#creat string
dbf, prj, shp, shx = [StringIO(zipfile.read(filename)) for filename in filenames]

r = shapefile.Reader(shp=shp, shx=shx, dbf=dbf)
print(r.numRecords)

#geopandas
attributes, geometry = [], []
field_names = [field[0] for field in r.fields[1:]]  
for row in r.shapeRecords():  
    geometry.append(shape(row.shape.__geo_interface__))  
    attributes.append(dict(zip(field_names, row.record)))      
print(row.shape.__geo_interface__)

#export projection
proj4_string = osr.SpatialReference(prj.read()).ExportToProj4()
print(proj4_string)

#construct GeoDataFrame
gdf = gpd.GeoDataFrame(data = attributes, geometry = geometry, crs = proj4)
print(gdf.head())

#extract county and print map
nj = gdf[gdf.STATEFP=='34'].to_crs(epsg = 3424)
ax = nj.plot(color = 'white', figsize=(10,10), linewidth=2)
ax.set(xticks=[], yticks=[])

plt.savefig("NJ_Counties.png", bbox_inches='tight')

NameError: name 'request' is not defined

In [1]:
import geopandas as gpd
import requests
import zipfile
import io
import matplotlib.pyplot as plt
%matplotlib inline

url = 'http://www2.census.gov/geo/tiger/GENZ2015/shp/cb_2015_us_county_500k.zip'
print('Downloading shapefile...')
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
print("Done")
# extract to folder
z.extractall(path='tmp/')

filenames = [y for y in sorted(z.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
print(filenames)

dbf, prj, shp, shx = [filename for filename in filenames]
usa = gpd.read_file(shp)
print("Shape of the dataframe: {}".format(usa.shape))
print("Projection of dataframe: {}".format(usa.crs))
usa.tail() #last 5 records in dataframe

ax = usa.plot()
ax.set_title("USA Counties. Default view)");

Done
['cb_2015_us_county_500k.dbf', 'cb_2015_us_county_500k.prj', 'cb_2015_us_county_500k.shp', 'cb_2015_us_county_500k.shx']


DriverError: cb_2015_us_county_500k.shp: No such file or directory